<a href="https://colab.research.google.com/github/TieuDiem/Stereo-Matching-Algorithm/blob/main/StereoMatching_From_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Declared distance method and lib

In [ ]:
import cv2
import numpy as np
from google.colab import drive
from enum import Enum
import os
from google.colab.patches import cv2_imshow
import math
from numpy.linalg import norm
import time

In [ ]:
def process_time(func):
    def wrap_func(*args, **kwargs):
        t1 = time.time()
        result = func(*args, **kwargs)
        t2 = time.time()
        print(f'Function {func.__name__!r} executed in {(t2-t1)*1000:.4f}ms')
        return result
    return wrap_func

In [ ]:
def distance_l1(x,y):
  return abs(x-y)

def distance_l2(x,y):
  return (x-y)**2

def distance_cosine_similarity(x,y):
  return np.dot(x,y)/ (norm(x)*norm(y))

def distance_correlation_cofficient(x,y):

  covariance=np.dot(x- [np.mean(x)] * len(x),y -[np.mean(y)] * len(y)) / len(x)
  correl = covariance / ( np.sqrt(np.var(x)) *  np.sqrt(np.var(y)))
  return correl

In [ ]:
distance_cosine_similarity(np.array([1,2,3,4]),np.array([1,2,3,4]))

In [ ]:
distance_correlation_cofficient(np.array([1,2,3,4]),np.array([2,4,6,8]))

#PROBLEM 1: pixel_wise_matching


In [ ]:
drive.mount('/content/drive')

In [ ]:
path_save = '/content/drive/MyDrive/EX_AIO_2022/DataExamAIO2022/Stereo_Matching/Aloe'

In [ ]:
def pixel_wise_matching(image_left:np.ndarray,
                        image_right:np.ndarray,
                        disparity_range:int,
                        compute_type,save_image =False):
  
  #---> astype float32 to compute
  image_left =image_left.astype(np.float32)
  image_right =image_right.astype(np.float32)

  #---> Get size and generate desth_map , scale disparity range
  height = image_left.shape[0]
  width =image_left.shape[1]
  
  depth_map =np.zeros((height,width),dtype=np.uint8)
  scale = 255 / disparity_range

  #---> Select mode of compute
  if  compute_type=="l1":
    distance = distance_l1
    print(f'Compute disparity map using pixel - wise matching with L1 ...')
  elif compute_type=="l2":
    print(f'Compute disparity map using pixel - wise matching with L2 ...')
    distance  = distance_l2

  #---> Calculate depth map 
  for y in range(1,height):
    for x in range(1,width):
        disparity = 0
        cost_min  = abs(image_left[y, x] - image_right[y, x])
        for d in range(1,disparity_range):
            if x-d <0 :
              cost =255 
            else : 
              cost =distance(image_left[y,x],image_right[y,x-d])
            if cost < cost_min:
                cost_min  = cost
                disparity = d
            continue
        depth_map[y, x] = disparity*scale  
        
    #---> Save image or not
  if save_image==True:
    cv2.imwrite(os.path.join(path_save,"image_pixel_matching.png") , depth_map)
    print(f'Saving result ...')
  print(f'Done')    
  return depth_map

In [ ]:
image_left =cv2.imread(os.path.join(path_save,"Aloe_left_1.png"),0)
image_right=cv2.imread(os.path.join(path_save,"Aloe_right_1.png"),0)

cv2_imshow(np.hstack((image_left,image_right)))

In [ ]:
desparity_image_L1 = pixel_wise_matching(image_left,image_right,16,compute_type="l1",save_image=True)
desparity_image_L2 = pixel_wise_matching(image_left,image_right,16,compute_type="l2",save_image=True)

In [ ]:
cv2_imshow(np.hstack((desparity_image_L1,desparity_image_L2)))
desparity_image_L1_color = cv2.applyColorMap(desparity_image_L1,cv2.COLORMAP_JET)

result = desparity_image_L1-desparity_image_L2
cv2_imshow(desparity_image_L1_color)

#PROBLEM 2: window-based-matching

In [ ]:
def window_base_matching(image_left:np.ndarray,
                         image_right:np.ndarray,
                         disparity_range,
                         compute_type,
                         kernel_size,
                         save_image =True):

  image_left  = image_left.astype(np.float32)
  image_right = image_right.astype(np.float32)

  height,width= image_left.shape[:2]
 
  depth_map = np.zeros((height,width),dtype =np.uint8)

  scale =255/ disparity_range

  if compute_type == "l1":
    max_value =255
    distance = distance_l1
    print(f'Compute disparity map using window-base-matching with L1 ...')
    
  elif compute_type == "l2":
    max_value =255**2
    distance =distance_l2
    print(f'Compute disparity map using window-base-matching with L2 ...')

  kernel_half = int((kernel_size-1)/2)

  for h in range(kernel_half,   height-kernel_half):
    for w in range(kernel_half, width-kernel_half): 
      disparity =0
      cost_min =  max_value
      for d in range(disparity_range):
        sum_cost =0
        for u in range(-kernel_half,kernel_half+1):
          for v in range(-kernel_half,kernel_half+1):
            if (w+v-d) >= 0 :
              cost = distance(int(image_left[h + u,w + v]),int(image_right[h + u,w+v-d]))
            else :
              cost = max_value
            sum_cost += cost                     
        if sum_cost < cost_min:
          cost_min = sum_cost
          disparity = d
        depth_map[h,w] = disparity*scale    
  if save_image==True:
    cv2.imwrite(os.path.join(path_save,"image_window_base_matching.png") , depth_map)
    print(f'Saving result ...')
  print(f'Done')    

  return depth_map

In [ ]:
desparity_image_L1 = window_base_matching(image_left,image_right,
                                          16,
                                          compute_type="l1", 
                                          kernel_size=5,
                                          save_image=True)

In [ ]:
desparity_image_L2 = window_base_matching(image_left,image_right,
                                          16,
                                          compute_type="l2", 
                                          kernel_size=5,
                                          save_image=True)

In [ ]:
cv2_imshow(np.hstack((desparity_image_L1,desparity_image_L2)))

# PROBLEM 3: Prove that a linear change is a noise

#PROBLEM 4,5: Stereo Matching with Cosine Simalarity, Correlation Cofficient

In [ ]:
def window_base_matching_v2(image_left,
                            image_right,
                            disparity_range,
                            compute_type,
                            kernel_size,
                            save_result=False):
  
  image_left =image_left.astype(np.float32)
  image_left =image_left.astype(np.float32)

  height =image_left.shape[0]
  width =image_left.shape[1]

  depth_map =np.zeros((height,width),dtype=np.uint8)
  scale = 255 / disparity_range

  kernel_half = int((kernel_size-1)/2)

  if compute_type =="cosine_simailarity":
    distance = distance_cosine_similarity
    name_image = "image_window_base_matching_cosine_similarity.png"
    print(f'Compute disparity map using window-base-matching with cosine similarity ...')
  elif compute_type=="correlation_cofficient":
    distance = distance_correlation_cofficient
    name_image = "image_window_base_matching_correlation_cofficient.png"
    print(f'Compute disparity map using window-base-matching with correlation cofficient ...')

  for h in range(kernel_half,height - kernel_half):
    for w in range(kernel_half,width -kernel_half):
      disparity =0
      cost_optimal = -1
      for j in range(disparity_range):
        d = w -j
        cost =-1
        if d-kernel_half >= 0:
          w_left = image_left [h-kernel_half:h+kernel_half+1 ,w-kernel_half:w+kernel_half+1  ]
          w_right = image_right [h-kernel_half: (h+kernel_half)+1 ,d-kernel_half: d+kernel_half+1  ]

          w_left=w_left.flatten()
          w_right=w_right.flatten()
          
          cost =distance(w_left,w_right)
          
        if cost > cost_optimal:
          cost_optimal=cost
          disparity =j
      depth_map[h,w] = disparity*scale
  if save_result==True:
    cv2.imwrite(os.path.join(path_save,name_image) , depth_map)
    print(f'Saving result ...')
  print(f'Done')    

  return depth_map

In [ ]:
image_left_new =cv2.imread('/content/drive/MyDrive/EX_AIO_2022/DataExamAIO2022/Stereo_Matching/Aloe/Aloe_left_1.png',0)
image_right_new =cv2.imread('/content/drive/MyDrive/EX_AIO_2022/DataExamAIO2022/Stereo_Matching/Aloe/Aloe_right_2.png',0)

In [ ]:
cv2_imshow(np.hstack((image_left_new,image_right_new)))

In [ ]:
cosine_similarity = window_base_matching_v2(image_left_new, image_right_new,
                                 64,
                                 "cosine_simailarity",
                                  3,True )

In [ ]:
correlation_cofficient = window_base_matching_v2(image_left_new,image_right_new,
                                 64,
                                 "correlation_cofficient",
                                  3,True )

In [ ]:
cv2_imshow(np.hstack((cosine_similarity,correlation_cofficient)))